In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import sklearn.model_selection
import torch

In [2]:
match_df = pd.read_csv('../output/csv/lol-data-matches-fixed-duration.csv')
frame_df = pd.read_csv('../output/csv/lol-data-match-frames.csv')
match_df = match_df.set_index('match_id')
match_df.drop(labels=['count','division','patch','region','first_rift_herald'], axis=1, inplace=True)
match_df.fillna(0, inplace=True)
match_df = match_df[match_df.winning_team!=0]
match_df.replace({
    'winning_team': {100: 1, 200: -1},
    'first_champion': {100: 1, 200: -1},
    'first_tower': {100: 1, 200: -1},
    'first_inhibitor': {100: 1, 200: -1},
    'first_baron': {100: 1, 200: -1},
    'first_dragon': {100: 1, 200: -1},
}, inplace=True)

match_df = match_df.astype({
    'winning_team': 'int32',
    'first_champion': 'int32',
    'first_tower': 'int32',
    'first_inhibitor': 'int32',
    'first_baron': 'int32',
    'first_dragon': 'int32',
})
match_df

,tier,game_duration,winning_team,first_champion,first_tower,first_inhibitor,first_baron,first_dragon
match_id,,,,,,,,
EUW1_5479661889,BRONZE,1797,1,1,1,1,1,1
EUW1_5479575964,BRONZE,1719,1,1,-1,1,0,1
EUW1_5479499524,BRONZE,1352,-1,-1,-1,-1,0,1
EUW1_5479492935,BRONZE,1647,-1,-1,1,-1,0,-1
EUW1_5479357161,BRONZE,1509,-1,-1,-1,-1,0,-1
...,...,...,...,...,...,...,...,...
EUW1_5544360421,GRANDMASTERS,1928,-1,1,-1,-1,-1,1
EUW1_5544335270,GRANDMASTERS,1101,-1,-1,-1,0,0,-1
EUW1_5544282724,GRANDMASTERS,1788,1,-1,1,1,1,-1


In [3]:
df = match_df.merge(frame_df,left_on='match_id', right_on='match_id').set_index('match_id')


In [4]:
# print(df.keys())
current_frame = 15

# Get the values of X and y for a given frame
X = df[df.frame==current_frame][[
    'blue_total_kills',
    'blue_total_gold',
    'blue_total_cs',
    'blue_total_damage',
    'blue_towers', 
    'blue_plates',
    'blue_inhibitors', 
    'blue_barons', 
    'blue_dragons', 
    'blue_rift_heralds',
    'red_total_kills', 
    'red_total_gold', 
    'red_total_cs', 
    'red_total_damage',
    'red_towers', 
    'red_plates', 
    'red_inhibitors', 
    'red_barons',
    'red_dragons', 
    'red_rift_heralds',
]].values
y = df[df.frame==current_frame]['winning_team'].values

# Convert -1 and +1 to 0 and 1 for the tensors
y = np.interp(y, (-1,+1), (0, 1)).astype(np.int32)

In [5]:
# Reshape the values one next to eachother
def combine(x):
    x = sklearn.preprocessing.normalize(x.reshape(2,10), norm='l1', axis=0)
    return x[0,:] - x[1,:]
# Normalize the data to have a set of 10 features that better correspond to a better comparison
X = np.apply_along_axis(combine, 1, X)

In [6]:
# Split the data 2/3 to 1/3
X_trn, X_tst, y_trn, y_tst = sklearn.model_selection.train_test_split(X, y, test_size=0.33, random_state=0)
# Scale the data with MinMax to avoid negative values
scaler = sklearn.preprocessing.MinMaxScaler()
scaler.fit(X_trn)
X_trn = scaler.transform(X_trn)
X_tst = scaler.transform(X_tst)


In [7]:
# Tensors setup
X_trn_torch = torch.tensor(X_trn, dtype=torch.float32)
y_trn_torch = torch.tensor(y_trn, dtype=torch.int64)
X_tst_torch = torch.tensor(X_tst, dtype=torch.float32)
y_tst_torch = torch.tensor(y_tst, dtype=torch.int64)

In [8]:
torch.manual_seed(0) # Ensure model weights initialized with same random numbers

# Create an object that holds a sequence of layers and activation functions
model = torch.nn.Sequential(
    torch.nn.Linear(10, 2)   # Applies Wx+b from 10 dimensions down to 2
)

# Create an object that can compute "negative log likelihood of a softmax"
loss = torch.nn.CrossEntropyLoss()

# Use stochastic gradient descent to train the model
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Use 100 training samples at a time to compute the gradient.
batch_size = 100

# Make 10 passes over the training data, each time using batch_size samples to compute gradient
num_epoch = 10
next_epoch = 1

In [9]:
for epoch in range(next_epoch, next_epoch+num_epoch):
    # Make an entire pass (an 'epoch') over the training data in batch_size chunks
    for i in range(0, len(X_trn), batch_size):        
        X_cur = X_trn_torch[i:i+batch_size]     # Slice out a mini-batch of features
        y_cur = y_trn_torch[i:i+batch_size]     # Slice out a mini-batch of targets
        
        y_pred = model(X_cur)                   # Make predictions (final-layer activations)
        l = loss(y_pred, y_cur)                 # Compute loss with respect to predictions
        
        model.zero_grad()                   # Reset all gradient accumulators to zero (PyTorch thing)
        l.backward()                        # Compute gradient of loss wrt all parameters (backprop!)
        optimizer.step()                    # Use the gradients to take a step with SGD.
        
    print("Epoch %2d: loss on final training batch: %.4f" % (epoch, l.item()))
    
print("Epoch %2d: loss on test set: %.4f" % (epoch, loss(model(X_tst_torch), y_tst_torch)))
next_epoch = epoch+1

Epoch  1: loss on final training batch: 0.4599
Epoch  2: loss on final training batch: 0.4412
Epoch  3: loss on final training batch: 0.4341
Epoch  4: loss on final training batch: 0.4294
Epoch  5: loss on final training batch: 0.4258
Epoch  6: loss on final training batch: 0.4230
Epoch  7: loss on final training batch: 0.4208
Epoch  8: loss on final training batch: 0.4190
Epoch  9: loss on final training batch: 0.4176
Epoch 10: loss on final training batch: 0.4165
Epoch 10: loss on test set: 0.4812
